<a href="https://colab.research.google.com/github/yuanjian-org/app/blob/main/tools/match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The algorithm for the initial matching in the [mentee-mentor matching process](https://mentors.org.cn/s/matchmaking). It finds the best many-to-many matching given mentee preferences, mentor capacity constraints, and other constraints.

## Input

`matching_scores_with_dummy.csv`: Mentee preference score matrics. A score ranges between -5 (least preferred) and 5 (most preferred). The dummy mentor is necessary only when mentor supply is lower than demand.

```
  mentee,MentorM,MentorN, ... ,Dummy
  MenteeE,0,-5, ... ,0
  MenteeF,2,4, ... ,0

```

`mentor_capacities_with_dummy.csv`: Max number of mentees a mentor is willing to take. All columns but `中文姓名` and `学生容量` are ignored.

```
  中文姓名,学生容量
  MentorM,2
  MentorN,4
  Dummy,3
```

## Output

```
  mentor=MentorM,assigned_mentees=MenteeE,MenteeF, ...
  mentor=MentorN,assigned_mentees=MenteeF, ...
  mentee=MenteeE,assigned_mentors=MentorM, ...
  mentor=MenteeF,assigned_mentors=MenteeM,MentorN ...
```  

In [ ]:
!pip3 install ortools

# Configuration

In [ ]:
import dataclasses
from typing import Dict, List, Mapping, Sequence, Tuple

import pandas as pd
import numpy as np
import numpy.ma as ma
import scipy.stats

# Constants.
# If true, remove mean and stdev.
_NORMALIZE_DATA = True

# If true, when removing mean and stdev, ignore the highest scores when
# computing the statistics and leave these scores unmodified.
_IGNORE_HIGHEST_SCORES = True
_HIGHEST_SCORE_CUTOFF = 3

# If true, for a mentee, among the mentors with scores higher than this,
# one of the mentors must be included in the matched pool.
MUST_MATCH_HIGHEST_SCORES = True
MUST_MATCH_SCORE_CUTOFF = _HIGHEST_SCORE_CUTOFF

# Key: Mentor capacity for final-matched mentees, Value: Mentor capacity for
# chat slots.
MENTOR_CAPACITY_UPPER_BOUND: Mapping[int, int] = {
    1: 4,
    2: 4,
    3: 2, # for dummy mentors. Dummies are necessary when mentor supply are lower than demand
}
DEFAULT_MENTOR_CAPACITY_UPPER_BOUND = 3
MENTEE_DEMAND_UPPER_BOUND = 3
MENTEE_DEMAND_LOWER_BOUND = 3

BANNED_SCORE = -5

# Higher offset encourages more matches. Setting to -10, for example,
# matches as few pairs as possible as long as the constraints are satisfied.
# This is the regularization term.
_SCORE_OBJ_OFFSET = 0.0

_SCORES_FILENAME = '/content/matching_scores_with_dummy.csv'
_CAPACITY_FILENAME = '/content/mentor_capacities_with_dummy.csv'
_CAPACITY_COLUMN_NAME = '学生容量'

In [ ]:
scores_df: pd.DataFrame = pd.read_csv(_SCORES_FILENAME, index_col=0)

mentors, mentor_names = zip(*enumerate(scores_df.columns.to_list()))
mentees, mentee_names = zip(*enumerate(scores_df.index.to_list()))
_MENTOR_ID_TO_NAME: Mapping[int, str] = {j: name for j, name in zip(mentors, mentor_names)}
_MENTEE_ID_TO_NAME: Mapping[int, str] = {i: name for i, name in zip(mentees, mentee_names)}

@dataclasses.dataclass(order=False, frozen=True)
class Pair:
  """Pair of mentor, mentee."""
  mentor: int
  mentee: int

scores_matrix: np.ndarray = scores_df.to_numpy(dtype=float)
if _NORMALIZE_DATA:
  if _IGNORE_HIGHEST_SCORES:
    scores_matrix_ma = ma.masked_where(
        (scores_matrix >= _HIGHEST_SCORE_CUTOFF) | (scores_matrix == BANNED_SCORE),
        scores_matrix, copy=False)
  else:
    scores_matrix_ma = ma.masked_where(
        scores_matrix == BANNED_SCORE, scores_matrix, copy=False)
  scores_matrix_ma -= scores_matrix_ma.mean(axis=1, keepdims=True)
  scores_matrix_ma /= scores_matrix_ma.std(axis=1, ddof=1, keepdims=True)
  scores_matrix = np.nan_to_num(scores_matrix)  # replace nan with 0
scores: Dict[Pair, float] = {
    Pair(mentee=i, mentor=j): scores_matrix[i][j]
    for i in range(len(mentees)) for j in range(len(mentors))}

print(f"mentors={mentor_names}")
print(f"mentees={mentee_names}")
if _NORMALIZE_DATA:
  print(f"###### NORMALIZED_SCORES ######\n{scores_matrix}")
print("###### SCORES TABLE ######")
scores_df

In [ ]:
capacity_df: pd.DataFrame = pd.read_csv(_CAPACITY_FILENAME, index_col='中文姓名')

mentor_id_to_capacity: Dict[str, int] = {}
for mentor, mentor_name in zip(mentors, mentor_names):
  mentor_id_to_capacity[mentor] = capacity_df.loc[mentor_name][_CAPACITY_COLUMN_NAME]
  print(f'{mentor_name=}, capacity={mentor_id_to_capacity[mentor]}')

# Solve

In [ ]:
from ortools.sat.python import cp_model


def ModelAssignment(
    mentors: Sequence[int],
    mentees: Sequence[int],
    scores: Mapping[Pair, float]
) -> Tuple[cp_model.CpModel, Mapping[Pair, cp_model.IntVar]]:
  """Models assignment problem and returns model and decision variables."""
  # Create the model.
  model = cp_model.CpModel()

  # Create the variables.
  pair_to_var: Mapping[Pair, cp_model.IntVar] = {}
  for pair, score in scores.items():
    # Binary decision variables, 1 if mentor is matched with mentee.
    upper_bound = 1 if scores[pair] != BANNED_SCORE else 0
    var = model.NewIntVar(
        0, upper_bound, f'mentor_{pair.mentor}_mentee_{pair.mentee}')
    pair_to_var[pair] = var

  # Create the constraints.
  mentor_to_vars: Mapping[int, Sequence[cp_model.IntVar]] = {
      mentor: [] for mentor in mentors}
  mentee_to_vars: Mapping[int, Sequence[cp_model.IntVar]] = {
      mentee: [] for mentee in mentees}
  mentee_to_must_match: Mapping[int, Sequence[cp_model.IntVar]] = {
      mentee: [] for mentee in mentees}
  for pair, var in pair_to_var.items():
    mentor_to_vars[pair.mentor].append(var)
    mentee_to_vars[pair.mentee].append(var)
    if MUST_MATCH_HIGHEST_SCORES and scores[pair] >= MUST_MATCH_SCORE_CUTOFF:
      mentee_to_must_match[pair.mentee].append(var)
  # Mentor capacity constraint: Every mentor paired with <= UB mentees.
  # Key: mentor ID. Value: all variables that contains to mentor ID.
  for mentor, vars in mentor_to_vars.items():
    model.Add(sum(vars) <= MENTOR_CAPACITY_UPPER_BOUND.get(
        mentor_id_to_capacity[mentor], DEFAULT_MENTOR_CAPACITY_UPPER_BOUND))
  # Mentee demand constraint: Every mentee paired with >= LB mentors
  # and <= UB mentors.
  # Key: mentor ID. Value: all variables that contains to mentor ID.
  for mentee, vars in mentee_to_vars.items():
    model.Add(sum(vars) >= MENTEE_DEMAND_LOWER_BOUND)
    model.Add(sum(vars) <= MENTEE_DEMAND_UPPER_BOUND)
  # For a mentee, among the mentors with scores higher than this,
  # one of the mentors must be included in the matched pool.
  for mentee, vars in mentee_to_must_match.items():
    if vars:
      model.Add(sum(vars) >= 1)

  # Create the objective.
  var_to_score = [(pair_to_var[pair], score + _SCORE_OBJ_OFFSET) for pair, score in scores.items()]
  objective_vars, objective_coefficients = zip(*var_to_score)
  model.Maximize(cp_model.LinearExpr.WeightedSum(
      expressions=objective_vars, coefficients=objective_coefficients))

  return model, pair_to_var

def SolveAssignment(model: cp_model.CpModel) -> Tuple[cp_model.CpSolver, str]:
  """Solves assignment problem."""
  # Creates a solver and solves the model.
  solver = cp_model.CpSolver()
  solver.parameters.random_seed = 123

  # Sets a time limit of 10 seconds.
  solver.parameters.max_time_in_seconds = 10.0

  status = solver.Solve(model)
  status_name = solver.StatusName(status)
  if status != cp_model.OPTIMAL:
    print(f"Non optimal solver status: {status_name=}")
  return solver, status_name


In [ ]:
# Create a constraint programming problem.
model, pair_to_var = ModelAssignment(
    mentors=mentors, mentees=mentees, scores=scores)
# Solve the problem.
solver, status_name = SolveAssignment(model=model)
# Print the solution.
print('\nStatistics')
print(f'  status   : {status_name}')
print(f'  conflicts: {solver.NumConflicts()}')
print(f'  branches : {solver.NumBranches()}')
print(f'  wall time: {solver.WallTime()} s')

if status_name != 'FEASIBLE' and status_name != 'OPTIMAL':
  raise ValueError(f'Unexpected: Solver state is {status_name}')
  
mentor_view: Mapping[int, Sequence[int]] = {mentor: [] for mentor in mentors}
mentee_view: Mapping[int, Sequence[int]] = {mentee: [] for mentee in mentees}
for pair, var in pair_to_var.items():
  assign_decision = solver.Value(var)
  if assign_decision == 1:
    mentor_view[pair.mentor].append(pair.mentee)
    mentee_view[pair.mentee].append(pair.mentor)
for mentor, assigned_mentees in mentor_view.items():
  print(
      f"mentor={_MENTOR_ID_TO_NAME[mentor]}, "
      f"assigned_mentees={[_MENTEE_ID_TO_NAME[mentee] for mentee in assigned_mentees]}")
for mentee, assigned_mentors in mentee_view.items():
  print(
      f"mentee={_MENTEE_ID_TO_NAME[mentee]}, "
      f"assigned_mentors={[_MENTOR_ID_TO_NAME[mentor] for mentor in assigned_mentors]}")

